In [6]:
import pandas as pd
import os

# Define the path to your xlsb file
file_path = os.path.join("../data", "Produits laitiers équitables - 2023.xlsb")


# Use pyxlsb to open the file and read it into a pandas dataframe
# Adjust the sheet name as necessary
df = pd.read_excel(file_path, engine='pyxlsb', sheet_name='Par SM')

# Remove the first five rows
df = df.iloc[6:]
df.head()
print(df)
#test

   Quantités vendues - année 2023 Unnamed: 1  Unnamed: 2  Unnamed: 3  \
6                           Baden         73        44.0        48.0   
7                            Bâle         52        34.0        45.0   
8                         Balerna         69        41.0       113.0   
9                          Bienne        478       537.0       527.0   
10                      Chavannes        902       927.0      1277.0   
11                           Chur         26        20.0        36.0   
12                       Delémont        295       325.0       336.0   
13                          Emmen         97        84.0       101.0   
14                       Fribourg       1147      1143.0      1359.0   
15                         Genève       1357      1287.0      1493.0   
16                       Lausanne       1133      1088.0      1600.0   
17                         Lugano        153       143.0       134.0   
18                          Marin       2204      2200.0      27

In [4]:
#rename columns
# Define the mapping of old column names to new column names
column_mapping = {
    'Quantités vendues - année 2023': 'Line Labels',
    'Unnamed: 1': '01/01/2023',
    'Unnamed: 2': '02/01/2023',
    'Unnamed: 3': '03/01/2023',
    'Unnamed: 4': '04/01/2023',
    'Unnamed: 5': '05/01/2023',
    'Unnamed: 6': '06/01/2023',
    'Unnamed: 7': '07/01/2023',
    'Unnamed: 8': '08/01/2023',
    'Unnamed: 9': '09/01/2023',
    'Unnamed: 10': '10/01/2023',
    'Unnamed: 11': '11/01/2023',
    'Unnamed: 12': '12/01/2023',
    'Unnamed: 13': 'Total General'
}

# Assuming df is your existing DataFrame
# Rename the columns using the provided mapping
df.rename(columns=column_mapping, inplace=True)

# Verify the change
df.head()



,Line Labels,01/01/2023,02/01/2023,03/01/2023,04/01/2023,05/01/2023,06/01/2023,07/01/2023,08/01/2023,09/01/2023,10/01/2023,11/01/2023,12/01/2023,Total General
6,Baden,73,44.0,48.0,33.0,57.0,40.0,30.0,42.0,50.226,42.000,55.000,57.000,571.226
7,Bâle,52,34.0,45.0,41.0,46.0,47.0,42.0,33.0,47.258,53.914,48.948,67.612,557.732
8,Balerna,69,41.0,113.0,111.0,81.0,64.0,61.0,87.0,98.778,91.000,106.000,141.000,1063.778
9,Bienne,478,537.0,527.0,645.0,462.0,408.0,493.0,433.0,523.010,435.000,572.000,485.000,5998.01
10,Chavannes,902,927.0,1277.0,1156.0,1127.0,984.0,1007.0,1065.0,956.862,1096.746,1227.246,1070.888,12796.742


In [16]:
#output_file_path = "cleaned_data.xlsx"
#df.to_excel(output_file_path, index=False)

In [5]:
df.loc[df['Line Labels'] == 'Saint-Gall Webersbleiche', 'Line Labels'] = 'St. Gall'
for i in df['Line Labels']:
    print(i)

Baden
Bâle
Balerna
Bienne
Chavannes
Chur
Delémont
Emmen
Fribourg
Genève
Lausanne
Lugano
Marin
Monthey
Morges
Nyon
Rapperswil
St. Gall
San Antonino
Sargans
Sierre
Sion
Vesenaz
Vevey
Vezia
Yverdon
Total général


In [6]:
data = {
    'City': df['Line Labels'].tolist(),
    'Total General': df['Total General'].tolist()
}
print(data)

{'City': ['Baden', 'Bâle', 'Balerna', 'Bienne', 'Chavannes', 'Chur', 'Delémont', 'Emmen', 'Fribourg', 'Genève', 'Lausanne', 'Lugano', 'Marin', 'Monthey', 'Morges', 'Nyon', 'Rapperswil', 'St. Gall', 'San Antonino', 'Sargans', 'Sierre', 'Sion', 'Vesenaz', 'Vevey', 'Vezia', 'Yverdon', 'Total général'], 'Total General': [571.226, 557.732, 1063.778, 5998.01, 12796.741999999998, 326.962, 3782.29, 1170.1860000000001, 14208.658, 15091.186, 14868.984, 1458.128, 26921.1, 39802.592000000004, 7940.012, 3370.256, 391.45799999999997, 489.73, 948.174, 431.076, 29548.824, 15158.846, 5026.776, 23123.688, 1679.6660000000002, 10287.003999999999, 238810.48599999998]}


In [7]:
# Convert the data into a DataFrame
df_map = pd.DataFrame(data)
df_map.head()

,City,Total General
0,Baden,571.226
1,Bâle,557.732
2,Balerna,1063.778
3,Bienne,5998.010
4,Chavannes,12796.742


In [8]:
import folium
from geopy.geocoders import Nominatim
import time
# Instantiate the geolocator
geolocator = Nominatim(user_agent="le_stores")

In [9]:
# Function to get latitude and longitude
def get_lat_lon(city):
    try:
        time.sleep(1)  # Simple rate-limiting mechanism
        location = geolocator.geocode(city + ', Switzerland')
        return location.latitude, location.longitude
    except AttributeError:
        return None, None

# Apply the function to get latitudes and longitudes
try:
    df_map[['Lat', 'Lon']] = df_map.apply(lambda row: pd.Series(get_lat_lon(row['City'])), axis=1)
    df_map = df_map[:-1]  # Remove the last row as before
except Exception as e:
    print(f"An error occurred: {e}")
print(df_map)

            City  Total General        Lat       Lon
0          Baden        571.226  47.473683  8.308682
1           Bâle        557.732  47.558108  7.587826
2        Balerna       1063.778  45.849191  9.007081
3         Bienne       5998.010  47.140208  7.243903
4      Chavannes      12796.742  46.528521  6.572670
5           Chur        326.962  46.854747  9.526490
6       Delémont       3782.290  47.361588  7.354445
7          Emmen       1170.186  47.075243  8.283946
8       Fribourg      14208.658  46.805566  7.161267
9         Genève      15091.186  46.204717  6.142311
10      Lausanne      14868.984  46.521827  6.632702
11        Lugano       1458.128  46.005010  8.952028
12         Marin      26921.100  47.009134  7.009020
13       Monthey      39802.592  46.252844  6.947826
14        Morges       7940.012  46.509327  6.498317
15          Nyon       3370.256  46.382058  6.240279
16    Rapperswil        391.458  47.226653  8.816404
17      St. Gall        489.730  47.425618  9.

In [10]:
# Create a map
m = folium.Map(location=[46.8182, 8.2275], zoom_start=7, tiles='Cartodb Positron', attr='Map data © OpenStreetMap contributors, CC-BY-SA, Imagery © Stamen Design') # Centered on Switzerland

# Add markers
for index, row in df_map.iterrows():
    folium.Marker(
        [row['Lat'], row['Lon']],
        popup=f"{row['City']}: Total General - {row['Total General']}",
    ).add_to(m)

# Display the map
m.save('swiss_cities_map.html')

In [11]:
import branca.colormap as cmp

In [12]:
# Create a linear color scale
max_sales = df_map['Total General'].max()
min_sales = df_map['Total General'].min()
color_scale = cmp.linear.viridis.scale(min_sales, max_sales)
color_scale.caption = 'Sales Intensity'

# Create a map with 'Stamen Terrain' tiles
m = folium.Map(location=[46.8182, 8.2275], zoom_start=7, tiles='Cartodb Positron',  attr='Map dat')

# Add markers with colors based on the linear color scale
for index, row in df_map.iterrows():
    if pd.notnull(row['Lat']) and pd.notnull(row['Lon']):
        folium.CircleMarker(
            location=[row['Lat'], row['Lon']],
            radius=5,  # Fixed radius for each marker
            popup=f"{row['City']}: Total General - {row['Total General']}",
            color=color_scale(row['Total General']),  # Color based on sales
            fill=True,
            fill_color=color_scale(row['Total General'])
        ).add_to(m)

# Add the color scale legend to the map
m.add_child(color_scale)

# Display the map
m.save('swiss_cities_map_gradient.html')


In [13]:
from IPython.display import IFrame

# Display the HTML file within the Jupyter Notebook using an IFrame
IFrame(src='swiss_cities_map_gradient.html', width=1000, height=600)

In [27]:
list(df_creme.columns)

['Row Labels',
 datetime.datetime(2023, 1, 1, 0, 0),
 datetime.datetime(2023, 2, 1, 0, 0),
 datetime.datetime(2023, 3, 1, 0, 0),
 datetime.datetime(2023, 4, 1, 0, 0),
 datetime.datetime(2023, 5, 1, 0, 0),
 datetime.datetime(2023, 6, 1, 0, 0),
 datetime.datetime(2023, 7, 1, 0, 0),
 datetime.datetime(2023, 8, 1, 0, 0),
 datetime.datetime(2023, 9, 1, 0, 0),
 datetime.datetime(2023, 10, 1, 0, 0),
 datetime.datetime(2023, 11, 1, 0, 0),
 datetime.datetime(2023, 12, 1, 0, 0),
 'Grand Total']

In [29]:
file_path = "data/creme_cafe_sales_per_stores_2023.xlsx"
df_creme = pd.read_excel(file_path)

column_mapping = {
    'Row Labels': 'Line Labels',
    '2023-01-01 00:00:00': '01/01/2023',
    'Unnamed: 2': '02/01/2023',
    'Unnamed: 3': '03/01/2023',
    'Unnamed: 4': '04/01/2023',
    'Unnamed: 5': '05/01/2023',
    'Unnamed: 6': '06/01/2023',
    'Unnamed: 7': '07/01/2023',
    'Unnamed: 8': '08/01/2023',
    'Unnamed: 9': '09/01/2023',
    'Unnamed: 10': '10/01/2023',
    'Unnamed: 11': '11/01/2023',
    'Unnamed: 12': '12/01/2023',
    'Unnamed: 13': 'Total General'
}

df_creme.rename(columns=column_mapping, inplace=True)
df_creme.head()

,Line Labels,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00,Grand Total
0,Ascona-Delta,29,21,46,78,53,69,78,49,47,36,31,30,567
1,Baden,31,19,21,8,25,11,18,20,24,26,33,22,258
2,Bâle,23,13,27,24,14,28,6,10,4,16,14,13,192
3,Balerna,9,7,11,15,4,5,10,12,27,16,22,28,166
4,Bienne,16,21,27,22,23,11,14,12,14,18,16,38,232
